In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import random
from scipy.optimize import fsolve
from scipy.optimize import minimize
import math
from scipy.stats import norm
from scipy.stats import truncnorm
from statistics import NormalDist
from scipy.integrate import quad
from scipy.integrate import dblquad
from scipy.integrate import tplquad
from itertools import product

In [2]:
from parameter import *

In [3]:
print("kappa_1: ", kappa_1)
print("kappa_2: ", kappa_2)
print("mu_xi_1: ", mu_xi_1)
print("mu_xi_2: ", mu_xi_2)
print("sigma_xi_1: ", sigma_xi_1)
print("sigma_xi_2: ", sigma_xi_2)
print("X_upper: ", X_upper)
print("X_lower: ", X_lower)
print("W_upper: ", W_upper)
print("W_lower: ", W_lower)

kappa_1:  0.5
kappa_2:  0.5
mu_xi_1:  0.03
mu_xi_2:  0
sigma_xi_1:  1.5
sigma_xi_2:  1.5
X_upper:  4
X_lower:  1
W_upper:  3
W_lower:  0


In [4]:
def monopoly(x, w, xi, omega):
    mc = np.exp(gamma*w + omega)
    T = np.exp(beta*x - alpha*mc + xi)

    #(1) find Y
    def monopoly_eqn(var):
        Y = var
        eq = 1 - Y + T*np.exp(-Y)
        return eq
    Y = fsolve(monopoly_eqn, 1)[0]
    
    pi = (1/alpha)*(Y-1) 
    price = Y/alpha + mc
    share = pi/(price-mc)

    return pi, price, share

In [5]:
def duopoly(x_1, x_2, w_1, w_2, xi_1, xi_2, omega_1, omega_2):
    try:
        mc_1 = np.exp(gamma*w_1 + omega_1) 
        mc_2 = np.exp(gamma*w_2 + omega_2)
        T_1 = np.exp(beta*x_1 - alpha*mc_1 + xi_1)
        T_2 = np.exp(beta*x_2 - alpha*mc_2 + xi_2)
        
        def duopoly_fun(Y):
            Y_1, Y_2 = Y
            eqn1 = Y_1 - math.log(T_1*(Y_2-1)) + math.log(1-Y_2+T_2*np.exp(-Y_2))        
            return abs(eqn1)
        
        def c1(Y):
            'Y_1 exp term greater than 0'
            Y_1, Y_2 = Y
            return 1-Y_1+T_1*np.exp(-Y_1)

        def c2(Y):
            'Y_2 exp term greater than 0'
            Y_1, Y_2 = Y 
            return 1-Y_2+T_2*np.exp(-Y_2)
        
        def c3(Y):
            Y_1, Y_2 = Y
            return Y_2 - math.log(T_2*(Y_1-1)) + math.log(1-Y_1+T_1*np.exp(-Y_1))

        bnds = ((1.0001, None), (1.0001, None))
        cons = ({'type': 'ineq', 'fun': c1}, 
                {'type': 'ineq', 'fun': c2},
                {'type': 'eq', 'fun': c3})
        initial_point = (1.0001, 1.0001)
        res = minimize(duopoly_fun, initial_point, method = 'SLSQP', bounds=bnds, constraints=cons)
        Y_1 = res.x[0]
        Y_2 = res.x[1]
        
        pi_1 = (1/alpha)*(Y_1-1)
        pi_2 = (1/alpha)*(Y_2-1)

        price_1 = Y_1/alpha + mc_1
        price_2 = Y_2/alpha + mc_2

        share_1 = pi_1/(price_1 - mc_1)
        share_2 = pi_2/(price_2 - mc_2)

        return pi_1, pi_2, price_1, price_2, share_1, share_2
    
    except:
        return 100, 100, 100, 100, 100, 100

In [6]:
def Base_DGP(market_num):
    
    x_1 = np.random.uniform(X_lower, X_upper, market_num)
    x_2 = np.random.uniform(X_lower, X_upper, market_num)

    w_1 = np.random.uniform(W_lower, W_upper, market_num)
    w_2 = np.random.uniform(W_lower, W_upper, market_num)
    
    
    # draw shocks 
    # firm1's brand equity is higher than firm2 
    xi_1 = np.random.normal(mu_xi_1, sigma_xi_1, market_num)
    xi_2 = np.random.normal(mu_xi_2, sigma_xi_2, market_num)   
    # omega_1 = np.random.uniform(-0.35, 0.05, market_num)   
    # omega_2 = np.random.uniform(-0.35, 0.05, market_num)   
    omega_1 = random.choices(population =[-0.05, 0.35], 
                             weights = [1/2, 1/2],
                             k = market_num)
    omega_2 = random.choices(population =[-0.05, 0.35], 
                             weights = [1/2, 1/2],
                             k = market_num)

    # generate a DF
    market_list = [i for i in range(1,market_num+1)]
    market_data = df({'market_num': market_list,
                      'x_1': x_1,
                      'x_2': x_2,
                      'w_1': w_1,
                      'w_2': w_2,
                      'xi_1': xi_1,
                      'xi_2': xi_2,
                      'omega_1': omega_1,
                      'omega_2': omega_2})

    return market_data

In [7]:
# generate a source data
MC_master_data = Base_DGP(market_num)

# create the identifier
# MC_master_data['job_array_num'] = np.array(range(1,market_num+1))

# save the data
# MC_master_data.to_csv("MC_master_data_0716.csv")
#MC_master_data.to_csv("MC_master_data.csv") 

In [8]:
vec_duopoly = np.vectorize(duopoly)

data_temp = MC_master_data[['x_1', 'x_2', 'w_1', 'w_2', 'xi_1', 'xi_2', 'omega_1', 'omega_2']].values
temp_duo = vec_duopoly(data_temp[:,0], data_temp[:,1], data_temp[:,2], data_temp[:,3], data_temp[:,4], data_temp[:,5], data_temp[:,6], data_temp[:,7])

In [ ]:
MC_master_data['duo_pi_1'] = temp_duo[0]
MC_master_data['duo_pi_2'] = temp_duo[1]
MC_master_data = MC_master_data.loc[MC_master_data['duo_pi_1'] != 100]
MC_master_data.shape[0]

14618

In [ ]:
MC_master_data['job_array_num'] = np.array(range(1,MC_master_data.shape[0]+1))
MC_master_data

,market_num,x_1,x_2,w_1,w_2,xi_1,xi_2,omega_1,omega_2,duo_pi_1,duo_pi_2,job_array_num
0,1,3.039229,1.350863,0.101988,2.545337,1.858982,-0.251418,0.35,0.35,4.511829,0.049610,1
1,2,2.808298,2.374881,0.313532,0.354326,0.294549,-0.105922,-0.05,0.35,1.829279,0.733523,2
2,3,1.732803,2.591669,2.143759,1.141474,-0.590730,-0.986291,0.35,0.35,0.191775,0.662305,3
3,4,1.001160,3.732682,0.456749,2.248021,-1.611330,-1.349456,-0.05,-0.05,0.132195,1.009949,4
4,5,2.187500,2.718428,0.392014,2.457749,0.571464,0.214862,0.35,0.35,1.531577,0.455515,5
...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14996,2.403173,2.026887,0.716086,1.019484,-0.041975,-0.372690,0.35,-0.05,1.037696,0.675260,14614
14996,14997,2.991575,1.086774,0.909493,0.360206,0.940498,-1.050989,0.35,0.35,2.724207,0.119462,14615
14997,14998,2.827171,1.825669,0.230094,2.383510,0.502838,-0.318862,-0.05,0.35,2.706252,0.118277,14616
14998,14999,2.221268,1.943933,2.337472,0.167315,1.000904,0.228242,0.35,-0.05,0.663090,1.237784,14617


In [ ]:
MC_master_data = MC_master_data.loc[MC_master_data['job_array_num'] <= 24000]

In [ ]:
MC_master_data.to_csv("MC_master_data.csv")

In [ ]:
import matplotlib.pyplot as plt